In [1]:
import json
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
data_file = open('goemotions.json')
data = np.array(json.load(data_file))

sentences = data[:, 0]
y_emotions = data[:, 1]
y_sentiments = data[:, 2]

# 2.1 Collecting the words and their frequencies

In [3]:
cv = CountVectorizer(analyzer='word')
X = cv.fit_transform(sentences)

In [ ]:
# size of the vocabulary
print(len(cv.vocabulary_))

# 2.2 Splitting the data in training and test sets

In [5]:
X_training, X_testing, y_training_emotions, y_testing_emotions, y_training_sentiments, y_testing_sentiments = train_test_split(X, y_emotions, y_sentiments, test_size=0.2, train_size=0.8, shuffle=False)

# 2.3.1 Base Multinomial Naive Bayes classifier

In [6]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifer_emotions = MultinomialNB()
mnb_classifer_emotions.fit(X_training, y_training_emotions)

mnb_classifer_sentiments = MultinomialNB()
mnb_classifer_sentiments.fit(X_training, y_training_sentiments)

MultinomialNB()

# 2.3.2 Base Decision Tree

In [8]:
from sklearn import tree
dt_classifer_emotions = tree.DecisionTreeClassifier()
dt_classifer_emotions.fit(X_training, y_training_emotions)

dt_classifer_sentiments = tree.DecisionTreeClassifier()
dt_classifer_sentiments.fit(X_training, y_training_sentiments)

DecisionTreeClassifier()

# 2.3.3 Multi-Layered Perceptron

In [22]:
from sklearn.neural_network import MLPClassifier

emotions_clf = MLPClassifier(max_iter=40)
emotions_clf.fit(X_training, y_training_emotions)

sentiments_clf = MLPClassifier()
sentiments_clf.fit(train_tokens, y_training_sentiments)

/home/adrien/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


NameError: name 'train_tokens' is not defined

# 2.3.4 Optimized Naive Bayes Classifier

In [12]:
# Creating NB classifiers with optimized alpha hyper params (2.3.4)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {"alpha": [0.5, 0, 2]}

mnb_classifer_emotions_new = MultinomialNB()
top_mnb_classifer_emotions = GridSearchCV(mnb_classifer_emotions_new, param_grid)
top_mnb_classifer_emotions.fit(X_training, y_training_emotions)

mnb_classifer_sentiments_new = MultinomialNB()
top_mnb_classifer_sentiments = GridSearchCV(mnb_classifer_sentiments_new, param_grid)
top_mnb_classifer_sentiments.fit(X_training, y_training_emotions)

/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/adrien/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will resul

GridSearchCV(estimator=MultinomialNB(), param_grid={'alpha': [0.5, 0, 2]})

In [15]:
print(top_mnb_classifer_emotions.best_estimator_)
print(top_mnb_classifer_sentiments.best_estimator_)

MultinomialNB(alpha=0.5)
MultinomialNB(alpha=0.5)


# 2.3.5 Top-DT

In [17]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [18]:
paramter_DT = {"criterion":["gini","entropy"],"max_depth":[5,10] , "min_samples_split":[2,5,10]}
clf_DecisionT = tree.DecisionTreeClassifier()
gsv_DT = GridSearchCV(clf_DecisionT, paramter_DT)

gsv_DT.fit(X_training,y_training_emotions) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...criterion=gini, max_depth=5, min_samples_split=2; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=2; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=2; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=2; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=2; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=5; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=5; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=5; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=5; total time=   3.5s
[CV] END ...criterion=gini, max_depth=5, min_samples_split=5; total time=   3.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=10; total time=   3.5s
[CV] END ..criterion=gini, max_depth=5, min_samp

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [5, 10],
                         'min_samples_split': [2, 5, 10]},
             verbose=2)

# 2.4 Performance Metrics

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Base-MNB (2.3.1)

In [29]:
# Emotions
mnb_classifer_emotions_results = mnb_classifer_emotions.predict(X_testing)
print(confusion_matrix(y_testing_emotions, mnb_classifer_emotions_results))
print(classification_report(y_testing_emotions, mnb_classifer_emotions_results))

[[ 929   18    0   12   30    3    1    3    0    2   10    1    0    7
     0   44    0   13   42    0  949   10    0    4    0    0    1   10]
 [  43  360    4   16   18    0    1    1    2    1    7    0    0    4
     0    6    0   14    8    0  704    3    0    2    0    0    0    4]
 [  15    5  113   76    6    4    2    5    0    6   25   12    0    1
     0    6    0    1    1    0  714    2    0    2    0    1    1    1]
 [  25   25   51  119   35    5    6    7    0   10   27   11    0    1
     0   18    0    1    2    0 1326    3    0    2    0    1    8    6]
 [ 124   15    6   29  220    4    7    4    3    4   31    3    0    2
     1   15    0    4   14    0 1715   12    0   12    0    1    4    1]
 [  18    1    0    5   25   40    1    4    0    4    8    0    0    1
     0   18    0    4    6    0  557   17    0    2    0    2    3    0]
 [   9    4    2   10   15    0   51   23    0    1   13    0    0    0
     0    7    0    1    0    0  782    1    0    9    0  

/home/adrien/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adrien/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

    admiration       0.47      0.44      0.46      2089
     amusement       0.53      0.30      0.38      1198
         anger       0.32      0.11      0.17       999
     annoyance       0.19      0.07      0.10      1689
      approval       0.25      0.10      0.14      2231
        caring       0.27      0.06      0.09       716
     confusion       0.31      0.05      0.09       934
     curiosity       0.42      0.10      0.17      1181
        desire       0.31      0.03      0.06       450
disappointment       0.23      0.03      0.06       950
   disapproval       0.25      0.08      0.12      1585
       disgust       0.45      0.09      0.15       622
 embarrassment       0.10      0.00      0.01       283
    excitement       0.29      0.04      0.08       600
          fear       0.52      0.04      0.07       364
     gratitude       0.75      0.67      0.71      1409
         grief       0.00      0.00      0.00  

/home/adrien/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
# Sentiments
mnb_classifer_sentiments_results = mnb_classifer_sentiments.predict(X_testing)
print(confusion_matrix(y_testing_sentiments, mnb_classifer_emotions_results))
print(classification_report(y_testing_sentiments, mnb_classifer_sentiments_results))

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [  91    0   28   19   47   67    5   74  163    2    6   42    2    0
     7    1   23    0   12    8    0 3076    9    0    0   47    0    3
    64]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0